In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
import pandas as pd
import numpy as np

# Load iris.csv
data = pd.read_csv("/content/drive/MyDrive/Concepts and technology of AI/Iris.csv")

# Drop 'Id' column
data = data.drop(columns=['Id'])

# Separate features and labels
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Convert string labels to integers manually
classes = np.unique(y)
class_to_int = {cls: idx for idx, cls in enumerate(classes)}
y_int = np.array([class_to_int[label] for label in y])

# One-hot encode labels manually
num_classes = len(classes)
y_onehot = np.eye(num_classes)[y_int]

# Manual train-test split
def train_test_split_manual(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)
    split = int(X.shape[0] * (1 - test_size))
    train_idx, test_idx = indices[:split], indices[split:]
    return X[train_idx], X[test_idx], y[train_idx], y[test_idx]

X_train, X_test, y_train, y_test = train_test_split_manual(X, y_onehot, test_size=0.2)

print("iris.csv loaded and split successfully!")
print("Training set size:", X_train.shape[0])
print("Test set size:", X_test.shape[0])
print("Classes:", classes)



iris.csv loaded and split successfully!
Training set size: 120
Test set size: 30
Classes: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


In [29]:
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def loss_softmax(y_true, y_pred):
    return -np.sum(y_true * np.log(y_pred + 1e-10)) / y_true.shape[0]



In [30]:
def compute_gradient_softmax(X, y, W, b):
    z = np.dot(X, W) + b
    y_pred = softmax(z)
    grad_W = np.dot(X.T, (y_pred - y)) / X.shape[0]
    grad_b = np.sum(y_pred - y, axis=0, keepdims=True) / X.shape[0]
    return grad_W, grad_b

def gradient_descent_softmax(X, y, W, b, alpha, n_iter, show_cost=False):
    cost_history = []
    for i in range(n_iter):
        grad_W, grad_b = compute_gradient_softmax(X, y, W, b)
        W -= alpha * grad_W
        b -= alpha * grad_b
        cost = loss_softmax(y, softmax(np.dot(X, W) + b))
        cost_history.append(cost)
        if show_cost and (i % 100 == 0 or i == n_iter - 1):
            print(f"Iteration {i}: Cost = {cost:.6f}")
    return W, b, cost_history


In [31]:
def predict_softmax(X, W, b):
    z = np.dot(X, W) + b
    y_pred = softmax(z)
    return np.argmax(y_pred, axis=1)

def accuracy_softmax(y_true, y_pred):
    return np.mean(y_true == y_pred)



In [32]:
def confusion_matrix_manual(y_true, y_pred, num_classes):
    cm = np.zeros((num_classes, num_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm


In [33]:
# Initialize parameters
n_features = X_train.shape[1]
W = np.zeros((n_features, num_classes))
b = np.zeros((1, num_classes))

# Train model
W, b, cost_history = gradient_descent_softmax(X_train, y_train, W, b, alpha=0.01, n_iter=1000, show_cost=True)

# Predictions
y_pred_train = predict_softmax(X_train, W, b)
y_pred_test = predict_softmax(X_test, W, b)

# Convert one-hot back to class labels
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# Accuracy
train_acc = accuracy_softmax(y_train_labels, y_pred_train)
test_acc = accuracy_softmax(y_test_labels, y_pred_test)

print("Training Accuracy:", train_acc)
print("Test Accuracy:", test_acc)

# Confusion Matrix
cm = confusion_matrix_manual(y_test_labels, y_pred_test, num_classes)
print("Confusion Matrix:\n", cm)



Iteration 0: Cost = 1.088282
Iteration 100: Cost = 0.693559
Iteration 200: Cost = 0.568868
Iteration 300: Cost = 0.507060
Iteration 400: Cost = 0.467758
Iteration 500: Cost = 0.439107
Iteration 600: Cost = 0.416470
Iteration 700: Cost = 0.397662
Iteration 800: Cost = 0.381512
Iteration 900: Cost = 0.367327
Iteration 999: Cost = 0.354788
Training Accuracy: 0.9583333333333334
Test Accuracy: 1.0
Confusion Matrix:
 [[ 7  0  0]
 [ 0 11  0]
 [ 0  0 12]]
